### Include libraries

In [8]:
require("Matrix")
require("rsvd")
require("factoextra")
require("foreach")
require("dplyr")
require("doParallel")

### Utility functions

In [9]:
calcRandPCA<-function(tmat, d=10, seed=42)
{
  # @param tmat A non-negative matrix with samples by features
  # @return A matrix with features by samples projected on PCA space
  set.seed(seed)
  
  rpca_obj <- rpca(tmat, k=d, center=T, scale=F, retx=T, p=10, q=7)
  rpca_obj$x
}

calcPDist <- function(tmat)
{
  get_dist(tmat,method = "pearson")
}

calcPContribution<-function(distMat)
{
  distMat<-as.matrix(distMat)
  rmat<-exp(-4.5*(distMat)^2)
  rmat[distMat>1]<-0
  diag(rmat)<-0
  
  return(Matrix(rmat, sparse=T))
}

findPNeighbors<-function(mat.pcontribution, positions, tau.p=16, nThreads=8)
{
  if(tau.p < 0)
    stop("Parameter tau.p must be 0 or positive.")
  
  bars<-positions$barcodes
  ind.bars<-1:length(bars)
  names(ind.bars)<-bars
  
  cl=makeCluster(nThreads)
  registerDoParallel(cl)
  
  p.neighbors<-foreach(i=1:length(bars),.packages = c("dplyr","Matrix")) %dopar%
    {
      a<-positions[mat.pcontribution[,i]>0,] 
      if(nrow(a)>0){
        b<-a %>% 
          mutate(p.contribution = mat.pcontribution[ind.bars[barcodes],i])
        if(0 == tau.p || nrow(a)<tau.p)
          b %>% mutate(ps.contribution = p.contribution/(sum(p.contribution)+0.00000001))
        else
          (b %>% arrange(desc(p.contribution)))[1:tau.p,] %>% mutate(ps.contribution = p.contribution/(sum(p.contribution)+0.00000001))
      } else
        NULL
    }
  
  stopCluster(cl)
  
  names(p.neighbors)<-bars
  
  return(p.neighbors)
  
}

#Spatial neighbors

GetPointNeighbors<-function(px,py,order=2,is.keepself=F)
{

  x.range<-(px-order):(px+order)
  x.range<-x.range[x.range>=0]
  y.range<-(py-order):(py+order)
  y.range<-y.range[y.range>=0]
  
  x.range<-rep.int(x.range,times=length(y.range))
  x.range<-sort(x.range)
  
  points<-data.frame(x=x.range, y=y.range)
  
  points<-points %>% mutate(distance=abs(x-px)+abs(y-py)) %>%
    filter(distance <= order)
  
  if(!is.keepself)
    points<-points %>% filter(distance>0)
  
  return(points)
  
}

findSNeighbors<-function(positions, is.hexa=F , tau.s=2, nThreads=8)
{
  if(is.hexa)
    tau.s<-tau.s*2

  bars<-positions$barcodes
  ind.bars<-1:length(bars)
  names(ind.bars)<-bars
  
  cl=makeCluster(nThreads)
  registerDoParallel(cl)
  
  s.neighbors<-foreach(i=1:length(bars),.packages =c("dplyr","Matrix"),.export = c("GetPointNeighbors")) %dopar%
    {
      a<-inner_join(x=positions, y=GetPointNeighbors(px=positions$st.x[i],py=positions$st.y[i],order = tau.s),by=c("st.x"="x","st.y"="y")) 
      if(nrow(a)>0){
        a %>%
        mutate(s.contribution=(1/distance)) %>%
        mutate(ss.contribution=s.contribution/(sum(s.contribution)+0.00000001))
      } else
        NULL
    } 
  
  stopCluster(cl)
  
  names(s.neighbors)<-positions$barcodes
  
  return(s.neighbors)
  
}


#get combined smoothed expression matrix

getCSmoothedExp<-function(exp, s.neighbors, p.neighbors, alpha, beta, nThreads=8)
{
  bars<-names(p.neighbors)
  ind.bars<-1:length(p.neighbors)
  names(ind.bars)<-bars

  cl=makeCluster(nThreads)
  registerDoParallel(cl)
  
  s.exp<-foreach(i=1:length(bars),.combine = "cbind",.packages = c("dplyr","Matrix")) %dopar%
    {
      if(!is.null(s.neighbors[[i]]))
        exp[,ind.bars[s.neighbors[[i]]$barcodes]] %*% Matrix(s.neighbors[[i]]$ss.contribution,sparse=T)
      else
        0
    }
  
  p.exp<-foreach(i=1:length(bars),.combine = "cbind",.packages = c("dplyr","Matrix")) %dopar%
    {
      if(!is.null(p.neighbors[[i]]))
        exp[,ind.bars[p.neighbors[[i]]$barcodes]] %*% Matrix(p.neighbors[[i]]$ps.contribution,sparse=T)
      else
        0
    }
  
  stopCluster(cl)
  
  exp.smoothed<-exp * (1-alpha) + (s.exp * beta + p.exp * (1-beta))*alpha

  return(exp.smoothed)
}

fillBlanks<-function(exp, coord, is.hexa=F, tau.s=2, filling.threshold = 0.5, nThreads = 8)
{

  x.bounds<-c(min(coord$st.x),max(coord$st.x))
  y.bounds<-c(min(coord$st.y),max(coord$st.y))
  x.availible<-x.bounds[1]:x.bounds[2]
  y.availible<-y.bounds[1]:y.bounds[2]
  x.availible<-sort(rep(x.availible,length(y.availible)))
  coord.maybe<-data.frame(st.x=x.availible,st.y=y.availible)

  if(is.hexa)
  {
    tau.s<-tau.s*2
    coord.maybe<- coord.maybe %>% 
      mutate(is.exist = ((st.x %% 2) == (st.y %% 2))) %>%
      select(st.x,st.y)
   }
  
  coord.maybe<-suppressMessages(coord %>% right_join(coord.maybe,copy = T) %>% filter(is.na(barcodes)))
  coord.maybe$barcodes<-paste0("SUPP",1:nrow(coord.maybe))
  
  cl=makeCluster(nThreads)
  registerDoParallel(cl)
  
  s.neighbors<-foreach(i=1:nrow(coord.maybe),.packages = c("dplyr"),.export = c("GetPointNeighbors")) %dopar%
    {
      nbs.maybe<-GetPointNeighbors(px = coord.maybe$st.x[i],py = coord.maybe$st.y[i],order = tau.s)
      if(is.hexa)
        nbs.maybe<-nbs.maybe %>% filter(0 == distance %% 2)
      
      nbs<-coord %>% inner_join(nbs.maybe,by=c("st.x"="x","st.y"="y"))
      if(nrow(nbs)>filling.threshold * nrow(nbs.maybe))
      {
        nbs %>%
        mutate(s.contribution=(1/distance)) %>%
        mutate(ss.contribution=s.contribution/(sum(s.contribution)+0.00000001))
      } else
      NULL
    }
  
  names(s.neighbors)<-coord.maybe$barcodes
  s.neighbors<-s.neighbors[sapply(s.neighbors,function(x){!is.null(x)})]
  
  if(0==length(s.neighbors))
  {
    s.exp<-NULL
    s.neighbors<-NULL
  }
  else
  {
    s.exp<-foreach(i=1:length(s.neighbors),.combine = "cbind",.packages = c("dplyr","Matrix")) %dopar%
      {
        exp[,s.neighbors[[i]]$barcodes] %*% Matrix(s.neighbors[[i]]$ss.contribution,sparse=T)
      }
    colnames(s.exp)<-names(s.neighbors)
  }
  stopCluster(cl)
  
  return(list(exp=s.exp,colData=filter(coord.maybe,barcodes %in% names(s.neighbors))))
}

selectSamples <- function(data.tbl, zero.cutoff=0.99) {
  print("Filter samples")
  print(sprintf("Number of samples %d", dim(data.tbl)[2]))
  keep.idx = colSums(data.tbl==0)<=(zero.cutoff*dim(data.tbl)[1])
  data.tbl = data.tbl[,keep.idx]
  print(sprintf("Remaining number of samples %d", dim(data.tbl)[2]))
  return(list("data"=data.tbl, "idx"=keep.idx))
}


# Remove genes with mostly zeros and low variances
# Filter out genes with more than zero.cutoff*100% zero expressed samples
# Filter our genes with lowest var.cutoff*100% variances
selectGenes <- function(data.tbl, zero.cutoff=0.5, var.cutoff=0.1) {
  print("Filter genes")
  print(sprintf("Number of genes %d", dim(data.tbl)[1]))
  keep.idx1 = rowSums(data.tbl==0)<=(zero.cutoff*dim(data.tbl)[2])
  data.tbl = data.tbl[keep.idx1,]
  keep.idx = keep.idx1
  print(sprintf("Remaining number of genes %d", dim(data.tbl)[1]))
  vars = apply(data.tbl, 1, var)
  if (var.cutoff > 0) {
    keep.idx2 = vars>=quantile(vars, var.cutoff)
    data.tbl = data.tbl[keep.idx2,]
    keep.idx = keep.idx[keep.idx2]
  }
  print(sprintf("Remaining number of genes %d", dim(data.tbl)[1]))
  return(list("data"=data.tbl, "idx"=keep.idx))
}


# Normalize the library size of each sample to sum up to scale
NormalizeLibrarySize <- function(data.tbl, scale=10000) {
  return(sweep(data.tbl, 2, colSums(data.tbl), FUN="/") * scale)
}

### Code

#### Read data

In [10]:
file_names <- sort(list.files(path='test_data/indiana/coords'))
out_path <- '/data/ani/datasets/indiana_gene_subset/counts_spcs/'

In [11]:
# Iterate over the files
for (file_name in file_names) {
    data<-read.csv(file = paste("test_data/indiana/counts/", file_name, sep=''), sep=",")
    coord<-read.csv(file = paste("test_data/indiana/coords/", file_name, sep=''), sep=",")
    # message(paste(NROW(data), NCOL(data)))
    # message(paste(NROW(coord), NCOL(coord)))

    # Select high variance genes
    gene.zero.cutoff = 0.7
    gene.var.cutoff = 0.0

    filtered.genes <- selectGenes(data, gene.zero.cutoff, gene.var.cutoff)
    data <- filtered.genes$data

    colnames = colnames(data)

    # Replace '.' in colnames with '-'
    colnames = gsub("\\.", "-", colnames)

    coord <- coord[colnames,]

    # Perform logCPM transformation
    data <- NormalizeLibrarySize(data)
    data <- log2(data+1)
    data[is.na(data)] <- 0

    # Set parameters
    tau.p=16
    tau.s=2
    alpha=0.6
    beta=0.4
    filling.threshold=0.5

    # If Visium slide, then set is.hexa to TRUE
    is.hexa=T

    # If padding is needed, set is.padding to TRUE
    is.padding=F

    # Find neighbors
    nThreads=8

    exp<-Matrix(as.matrix(data))

    rpca.res <- calcRandPCA(t(exp)) 
    pdist.mat <- calcPDist(rpca.res)  
    rmat <- calcPContribution(pdist.mat)

    positions <- coord

    colnames(positions) <- c("st.x","st.y")
    positions$barcodes <- rownames(positions)

    # Pattern neighbors
    p.neighbors <- findPNeighbors(rmat, positions, tau.p, nThreads)
    # Spatial neighbors
    s.neighbors <- findSNeighbors(positions, tau.s, nThreads)

    # Smoothing
    bars<-names(p.neighbors)
    ind.bars<-1:length(p.neighbors)
    names(ind.bars)<-bars

    cl=makeCluster(nThreads)
    registerDoParallel(cl)
    
    # Spatially smoothed expression matrix
    s.exp<-foreach(i=1:length(bars),.combine = "cbind",.packages = c("dplyr","Matrix")) %dopar%
    {
      if(!is.null(s.neighbors[[i]]))
        exp[,ind.bars[s.neighbors[[i]]$barcodes]] %*% Matrix(s.neighbors[[i]]$ss.contribution,sparse=T)
      else
        0
    }

    # Pattern smoothed expression matrix
    p.exp<-foreach(i=1:length(bars),.combine = "cbind",.packages = c("dplyr","Matrix")) %dopar%
    {
      if(!is.null(p.neighbors[[i]]))
        exp[,ind.bars[p.neighbors[[i]]$barcodes]] %*% Matrix(p.neighbors[[i]]$ps.contribution,sparse=T)
      else
        0
    }

    # Calculate smoothed expression matrix
    stopCluster(cl)

    data.combinedsmooth <- exp * (1-alpha) + (s.exp * beta + p.exp * (1-beta))*alpha

    if(is.padding)
    {
        filled.blanks<-fillBlanks(data.combinedsmooth, positions, is.hexa, tau.s, filling.threshold, nThreads)

        data.combinedsmooth = as.data.frame(as.matrix(cbind(data.combinedsmooth, filled.blanks$exp)))
        
        attr(data.combinedsmooth,"SUPP_ColData")<-filled.blanks$colData
    }
    else
    {
        data.combinedsmooth = as.data.frame(as.matrix(data.combinedsmooth))
    }
    
    if (sum(is.na(data.combinedsmooth)) > 0) {
        mean.exp = sum(data.combinedsmooth[!is.na(data.combinedsmooth)]) / sum(!is.na(data.combinedsmooth))
        data.combinedsmooth[is.na(data.combinedsmooth)] = mean.exp
    }

    # Write the combined smoothed expression matrix to a file
    write_path = paste(out_path, file_name, sep='')
    write.csv(data.combinedsmooth, file = write_path, row.names = TRUE)
    
}

[1] "Filter genes"
[1] "Number of genes 36591"
[1] "Remaining number of genes 6656"
[1] "Remaining number of genes 6656"
[1] "Filter genes"
[1] "Number of genes 36591"
[1] "Remaining number of genes 1894"
[1] "Remaining number of genes 1894"
[1] "Filter genes"
[1] "Number of genes 36591"
[1] "Remaining number of genes 3799"
[1] "Remaining number of genes 3799"
[1] "Filter genes"
[1] "Number of genes 36591"
[1] "Remaining number of genes 4217"
[1] "Remaining number of genes 4217"
